---
title: "Child Mortality, Fertility, and Employment"  
author: "James Matosse"
format: 
    dashboard:
        theme: yeti
        echo: false
#        include: true
        fig.align: center
        layout:
            fill: true
---

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from itables import show
import country_converter as coco
from IPython.display import Markdown

import warnings
# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# Load the data
child_mortality = pd.read_csv(
    "data/child_mortality_0_5_year_olds_dying_per_1000_born.csv"
)
fertility_wide = pd.read_csv("data/children_per_woman_total_fertility.csv")
employ_wide = pd.read_csv("data/females_aged_15plus_employment_rate_percent.csv")

## data cleaning
def melt_data(df, value_name):
    return df.melt(id_vars="country", var_name="year", value_name=value_name)

child_mortality_long = melt_data(child_mortality, "mortality")
fertility_long = melt_data(fertility_wide, "woman_fertility")
employ_long = melt_data(employ_wide, "employment_rate")

replacements = {"UAE": "United Arab Emirates", "Channel Islands": "United Kingdom"}
for df in [child_mortality_long, fertility_long, employ_long]:
    df["country"] = df["country"].replace(replacements)
    df["year"] = df["year"].astype(int)

# Merge datasets
child_fertility = pd.merge(
    child_mortality_long, fertility_long, on=["country", "year"], how="left"
)

# Filter for analysis years
filtered_child_fertility = child_fertility.query("1991 <= year <= 2019")

# Convert 'country' to ISO3 country codes
filtered_child_fertility["country_code"] = coco.convert(
    names=filtered_child_fertility["country"], to="ISO3"
)
employ_long["country_code"] = coco.convert(
    names=employ_long["country"], to="ISO3"
)

# Merge datasets on 'country_code' and 'year'
employ_child_fert = pd.merge(
    filtered_child_fertility,
    employ_long,
    on=["country_code", "year"],
    how="left",
)

# Drop duplicate 'country_y' column and rename 'country_x' to 'country'
employ_child_fert = employ_child_fert.drop(columns="country_y").rename(
    columns={"country_x": "country"}
)

# Assign continents based on country names
employ_child_fert["continent"] = employ_child_fert["country"].apply(
    lambda x: coco.convert(names=x, to="continent")
)

# Reorder columns for logical grouping
col_list = list(employ_child_fert.columns)
col_list.insert(1, col_list.pop(4))  # Move 'country_code' to 2nd position
col_list.insert(2, col_list.pop(6))  # Move 'continent' to 3rd position
employ_child_fert = employ_child_fert[col_list]

# Remove rows with missing values
employ_child_fert = employ_child_fert.dropna()

In [ ]:
## Data Manipulation and Visualization
# Subset the data to include `country`, `1999`, and `2019` columns only.
table_df = child_mortality[["country", "1999", "2019"]]
table_df = table_df.rename(columns={"country": "Country/area"})

# Calculate absolute change as the difference between 2019 and 1999
table_df["Absolute Change"] = table_df["2019"] - table_df["1999"]

# Calculate relative change as a percentage
table_df["Relative Change"] = (
    (table_df["2019"] - table_df["1999"]) / table_df["1999"]
) * 100

# Round to 0 decimal places, and add a % sign to the relative change
table_df["Relative Change"] = table_df["Relative Change"].round(0).astype(str) + "%"

# Define a consistent color mapping for continents and countries
country_color_map = {
    "China": "#d62728",
    "Egypt": "#1f77b4",
    "Australia": "#2ca02c",
    "Argentina": "#ff7f0e",
    "Germany": "#9467bd",
}

continent_color = {
    "Africa": "#1f77b4",
    "America": "#ff7f0e",
    "Oceania": "#2ca02c",
    "Asia": "#d62728",
    "Europe": "#9467bd",
}

# Convert mortality rate to percentage for consistency in scaling (if required elsewhere)
employ_child_fert["mortality_pct"] = employ_child_fert["mortality"] / 10

# Find the country with the lowest mortality per continent
lowest_mortality = employ_child_fert.loc[
    employ_child_fert.groupby("continent")["mortality"].idxmin()
].sort_values("mortality")

# Bar chart for lowest child mortality by continent
fig_bar = px.bar(
    lowest_mortality,
    x="continent",
    y="mortality",
    color="continent",
    hover_name="country"
    title="Lowest Child Mortality per Continent (2019)",
    labels={
        "mortality": "Child Mortality",
        "continent": "Continent",
        "country": "Country",
    },
    color_discrete_map=continent_color,
    text_auto=".1f",
)

# Animated map of child mortality
map_fig = px.choropleth(
    employ_child_fert,
    locations="country_code",
    color="mortality",
    hover_name="country",
    animation_frame="year",
    title="Global Child Mortality (1991–2019)",
    color_continuous_scale="Reds",
    labels={
        "country_code": "Country Code",
        "mortality": "Child Mortality",
        "year": "Year",
    },
).update_layout(
    coloraxis_showscale=True
)  # Add scale visibility


fig_fert = px.line(
    employ_child_fert.query(
        'country in ["China", "Egypt", "Australia", "Argentina", "Germany"]'
    ),
    x="year",
    y="woman_fertility",
    color="country",
    title="Women's Fertility Trends (1991–2019)",
    markers=True,
    color_discrete_map=country_color_map,
    labels={
        "country": "Country",
        "woman_fertility": "Total Fertility Rate (TFR)",
        "year": "Year",
    },
).update_layout(
    showlegend=True,
    xaxis_title="Year",
    yaxis_title="Total Fertility Rate",
)

avg_fert_cont = (
    employ_child_fert.groupby("continent")
    .agg(mean_fertility=("woman_fertility", "mean"))
    .reset_index()
    .sort_values(by="mean_fertility", ascending=False)
)

fig_fert1 = px.pie(
    avg_fert_cont,
    names="continent",
    values="mean_fertility",
    color="continent",
    title="Average Fertility per Continent (1991–2019)",
    labels={"mean_fertility": "Average Fertility", "continent": "Continent"},
    hover_data=["mean_fertility"],
    color_discrete_map=continent_color,
).update_traces(textinfo="label+percent", showlegend=False)

lowest20 = (
    employ_child_fert[employ_child_fert["year"] == 2019]
    .sort_values("woman_fertility", ascending=True)
    .head(20)
)

fig_fert2 = px.bar(
    lowest20,
    y="country",
    x="woman_fertility",
    color="continent",
    hover_name="continent",
    title="Lowest Total Fertility Rates in 2019",
    labels={
        "continent": "Continent",
        "woman_fertility": "Total Fertility Rate",
        "country": "Country",
    },
    color_discrete_map=continent_color,
    text_auto=".2f",
).update_layout(
    showlegend=False,
    xaxis_title="Total Fertility Rate",
    yaxis_title="Country",
)

# Create the map using plotly express with a green color scale
map_fig2 = px.choropleth(
    employ_child_fert,
    locations="country_code",
    color="employment_rate",
    hover_name="country",
    animation_frame="year",
    title="Global Female Employment Rate (1991–2019)",
    labels={
        "country_code": "Country Code",
        "employment_rate": "Employment Rate (%)",
        "year": "Year",
    },
    color_continuous_scale="Greens",
).update_layout(
    coloraxis_showscale=True,
)


# Home page information
def assign_employ_group(employ_rate):
    """
    Categorizes employment rates into four groups:
    - 3-35%: Low employment
    - 36-46%: Moderate employment
    - 47-57%: High employment
    - 58-90%: Very high employment
    """
    if employ_rate >= 3 and employ_rate < 36:
        return "3-35"
    elif employ_rate >= 36 and employ_rate < 47:
        return "36-46"
    elif employ_rate >= 47 and employ_rate < 58:
        return "47-57"
    elif employ_rate >= 58:
        return "58-90"
    else:
        return "Invalid"


# assign_employ_group(20)
assign_employ_groupVec = np.vectorize(assign_employ_group)

employ_child_fert["employ_group"] = assign_employ_groupVec(
    employ_child_fert["employment_rate"]
)

# Create summary table for employ groups
employ_mortality_table = (
    employ_child_fert.groupby("employ_group")
    .agg(
        mean_mortality_employ=("mortality_pct", "mean"),
        num_inds=("mortality_pct", "size"),
    )
    .reset_index()
)

fig_mix = (
    px.bar(
        employ_mortality_table,
        x="employ_group",
        y="mean_mortality_employ",
        title="Average Child Mortality per Grouped Employment Rate",
        labels={
            "mean_mortality_employ": "Child Mortality",
            "employ_group": "Employment Rate Group",
        },
        text_auto=".1f",
    )
    .update_layout(
        showlegend=False,
        xaxis_title="Female Employment Rate Group (%)",
        yaxis_title="Child Mortality (%)",
    )
    .update_traces(
        marker_color="#2ca02c",
        text=employ_mortality_table["mean_mortality_employ"].round(1),
        textposition="outside",
    )
)

fig_mix1 = px.scatter(
    employ_child_fert,
    y="mortality_pct",
    x="woman_fertility",
    animation_frame="year",
    color="continent",
    title="Child Mortality vs. Woman's fertility (bubble size: Female Employment Rate)",
    size="employment_rate",
    hover_name="country",
    color_discrete_map=continent_color,
    labels={
        "country": "Country",
        "woman_fertility": "Woman's Fertility",
        "year": "Year",
        "mortality_pct": "Child Mortality",
        "continent": "Continent",
        "employment_rate": "Employment Rate",
    },
).update_layout(
    showlegend=False,
    xaxis_title="Total Fertility Rate",
    yaxis_title="Child Mortality (%)",
)

# Compute highest female employment rate
highestE_row = (
    employ_child_fert.sort_values("employment_rate", ascending=False).head(1).squeeze()
)
highest_employ_rat_country = highestE_row["country"]
highest_employ_rat_value = round(highestE_row["employment_rate"], 2)
highest_employ_rat_year = highestE_row["year"]


# Compute lowest child mortality
lowest_row = (
    employ_child_fert.sort_values("mortality", ascending=True).head(1).squeeze()
)
lowest_mortality_country = lowest_row["country"]
lowest_mortality_value = round(lowest_row["mortality"], 1)
lowest_mortality_year = lowest_row["year"]


# Compute highest total fertility rate
highestF_row = (
    employ_child_fert.sort_values("woman_fertility", ascending=False).head(1).squeeze()
)
highest_fertility_country = highestF_row["country"]
highest_fertility_value = round(highestF_row["woman_fertility"], 2)
highest_fertility_year = highestF_row["year"]

# Home Page

## Row 1 {height=35%}

::: {.valuebox icon="heart" color="#1E90FF" title="Lowest Child Mortality"}

`{python} str(lowest_mortality_value)`

`{python} lowest_mortality_country` in `{python} str(lowest_mortality_year)`

:::

::: {.valuebox icon="flower3" color="#87CEFA" title="Highest Total Fertility Rate"} 

`{python} str(highest_fertility_value)`

`{python} highest_fertility_country` in `{python} str(highest_fertility_year)`

:::

::: {.valuebox icon="mortarboard-fill" color="#4682B4" title="Highest Female Employment Rate"}

`{python} str(highest_employ_rat_value)`

`{python} highest_employ_rat_country` in `{python} str(highest_employ_rat_year)`

:::

## Row 2 {height=60%}

### Comumn {width="60%"}


In [ ]:
fig_mix1

### Column 


In [ ]:
fig_mix

# Child Mortality 

## Row {height=80%}

### Column {width="60%"}


In [ ]:
Markdown(table_df.to_markdown())

### Column {.tabset width="40%"}

#### Map


In [ ]:
map_fig.show()

#### Chart


In [ ]:
fig_bar

## Row {height="20%"}

The chart highlights clear disparities between continents in child mortality rates, with developed regions (Europe, Asia, Oceania) showing significantly lower rates compared to developing regions (America and Africa).

This data underscores the importance of healthcare infrastructure, economic development, and public health initiatives in reducing child mortality rates. Finland having the lowest child mortality rate among all continents, with a value close to 1. This suggests excellent healthcare systems and child welfare practices in the country. In contrast, Libya has the highest child mortality rate among the listed continents, with a value significantly higher than the rest. This reflects disparities in healthcare access, infrastructure, and socioeconomic conditions in the African region.

# Woman's Fertility {orientation="columns"}

## Column 1 {width="60%"}

### Row {.tabset height=80%}

#### Pie


In [ ]:
fig_fert1

#### line


In [ ]:
fig_fert

### Row

There is a clear divide between continents with higher fertility rates (Africa and Oceania) and those with lower rates (Europe, America, and parts of Asia). The data reflects varying levels of development, healthcare access, education, and cultural norms influencing fertility rates worldwide.

## Column 2 


In [ ]:
fig_fert2

# Female Employment Rate

## Row 1 {height=50%}


In [ ]:
continent_fig = px.violin(
    employ_child_fert,
    x="continent",
    y="employment_rate",
    color="continent",
    animation_frame="year",
    labels={
        "continent": "Continent",
        "employment_rate": "Employment Rate (%)",
        "year": "Year",
        "country": "Country",
    },
    title="Female Employment Rate Distribution by Continent (1991–2019)",
    points="all",
    hover_data=["country"],
    color_discrete_map=continent_color,
    box=True,
).update_layout(
    showlegend=False,
    xaxis_title="Continent",
    yaxis_title="Employment Rate",
    sliders=[
        {
            "active": employ_child_fert["year"]
            .unique()
            .tolist()
            .index(2018),  # Set slider to 2018 index
        }
    ],
)

# Ensure the animation starts at 2018 by setting the default frame
continent_fig.frames = [
    frame for frame in continent_fig.frames if frame.name == "2018"
] + [
    frame for frame in continent_fig.frames if frame.name != "2018"
]

# Show the plot
continent_fig

## Row 2 {height=50%}

### Column {width="60%"}


In [ ]:
map_fig2

### Column {width="40%"}

**For 2018** Africa and Oceania have relatively high upper bounds for female employment rates compared to other continents. In contrast, Asia has the lowest minimum values, indicating that some countries have significantly low participation rates.

Africa exhibits the widest range of female employment rates, reflecting significant variability between countries on this continent. However, it also has the highest median, suggesting that most countries in Africa have a relatively high rate of female employment compared to other continents.

Asia also shows a broad range, with some countries having very low employment rates and others quite high. On the other hand, Europe, America, and Oceania demonstrate relatively smaller ranges, indicating more consistency within these continents. Furthermore, Europe and America have lower medians compared to Africa and Oceania, reflecting a slightly lower overall female employment rate.

# Data Download


In [ ]:
employ_child_fert = employ_child_fert.rename(
    columns={
        "country": "Country/area",
        "country_code": "Country Code",
        "continent": "Continent",
        "year": "Year",
        "mortality": "Child Mortality",
        "woman_fertility": "Fertility Rate",
        "employment_rate": "Employment Rate",
    }
)
employ_child_fert = employ_child_fert.drop(columns=['mortality_pct', 'employ_group','Country Code'])

# Column descriptions
column_descriptions = """
- Country/area: The name of the country or area.
- Country Code: The ISO code for the country.
- Continent: The continent where the country is located.
- Year: The year of observation.
- Child Mortality: Number of deaths under age 5 per 1000 live births.
- Fertility Rate: Number of children a woman would have in her lifetime based on current age-specific fertility rates.
- Employment Rate: Percentage of females aged 15+ employed.
"""

# print(column_descriptions)

# Display full dataset with itables
show(
    employ_child_fert,
    buttons=["csvHtml5"],
)

# About

The data used for this PBB's capstone project was obtained from [Gapminder's data repository](https://gapminder.org/data). Three indicators were selected for analysis namely; "child mortality", "total fertilty rate", and "female employment rate".

Child mortality is for 0 to 5 year olds dying per 1000 born.
Total fertility rate is the number of children that would be born to a woman if she were to live to the end of her childbearing years and bear children in accordance with age-specific fertility rates of the specified year.
Female employment rate is the percentage of female population, age group 15+, that has been employed during the given year.

Home page shows an overview of the analysis:
The **bar** plot shows that most of the countries with the female employment rate of 58% to 90% have the highest average of child mortality, followed by the lowest employment group. This might be due to most developing countries in Africa and some in Asia.
The **scatter** plot shows that most African countries had high child mortality and woman's fertility in 1991 with few Asian countries. In 2019, child mortality has reduced by half from about 30% in 1991 yet the number of babies per woman are still above 3 for most African countries.

The Gapminder Foundation is a non-profit venture that promotes sustainable global development and achievement of the United Nations Millennium Development Goals by increasing use and understanding of statistics and other information about social, economic, and environmental development.

**Watch Hans Rosling on Child Mortality in Developing Countries:**

<iframe width="560" height="315" src="https://youtube.com/embed/OwII-dwh-bk?si=iVzuHDVHmiGmM_6b" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>